In [199]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn import metrics
%matplotlib inline

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [200]:
data = pd.read_csv('./House_Rent_Dataset.csv')
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


### Предварительная обработка данных
В первую очередь проверим есть ли пропущенные значения

In [201]:
data.isna().sum()

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

Пропущенных значений нет

In [202]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [203]:
# у столбца Posted On указан тип object, преобразуем его во время.
data['Posted On'] = pd.to_datetime(data['Posted On'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Posted On          4746 non-null   datetime64[ns]
 1   BHK                4746 non-null   int64         
 2   Rent               4746 non-null   int64         
 3   Size               4746 non-null   int64         
 4   Floor              4746 non-null   object        
 5   Area Type          4746 non-null   object        
 6   Area Locality      4746 non-null   object        
 7   City               4746 non-null   object        
 8   Furnishing Status  4746 non-null   object        
 9   Tenant Preferred   4746 non-null   object        
 10  Bathroom           4746 non-null   int64         
 11  Point of Contact   4746 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 445.1+ KB


In [204]:
[(data.columns[i], data.iloc[:, i].unique().shape[0]) for i in range(len(data.columns))]

[('Posted On', 81),
 ('BHK', 6),
 ('Rent', 243),
 ('Size', 615),
 ('Floor', 480),
 ('Area Type', 3),
 ('Area Locality', 2235),
 ('City', 6),
 ('Furnishing Status', 3),
 ('Tenant Preferred', 3),
 ('Bathroom', 8),
 ('Point of Contact', 3)]

В столбце `Area Locality`, где хранится информация о местоположении дома, слишком много уникальных значений.

In [205]:
data = data.drop(['Area Locality', 'Posted On', 'Floor'],axis=1)

In [206]:
columns = ['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']
data = pd.get_dummies(data, columns=columns)

data.head()

,BHK,Rent,Size,Bathroom,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,City_Chennai,City_Delhi,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,10000,1100,2,False,False,True,False,False,False,...,False,False,False,True,False,True,False,False,False,True
1,2,20000,800,1,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,2,17000,1000,1,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,True
3,2,10000,800,1,False,False,True,False,False,False,...,False,False,False,True,False,True,False,False,False,True
4,2,7500,850,1,False,True,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True


In [207]:

data.corr()['Rent'].sort_values(ascending=False)

Rent                                 1.000000
Bathroom                             0.441215
Size                                 0.413551
BHK                                  0.369718
Point of Contact_Contact Agent       0.339750
City_Mumbai                          0.327038
Area Type_Carpet Area                0.215769
Furnishing Status_Furnished          0.110576
Tenant Preferred_Family              0.063941
Furnishing Status_Semi-Furnished     0.045309
Tenant Preferred_Bachelors           0.042151
Point of Contact_Contact Builder    -0.005482
Area Type_Built Area                -0.006439
City_Delhi                          -0.027072
City_Bangalore                      -0.061512
Tenant Preferred_Bachelors/Family   -0.078774
City_Chennai                        -0.082361
City_Hyderabad                      -0.087465
City_Kolkata                        -0.105322
Furnishing Status_Unfurnished       -0.126271
Area Type_Super Area                -0.215499
Point of Contact_Contact Owner    

### Разбиваем данные и масштабируем данные

In [208]:
# Разделение на признаки и целевую переменную
X = data.drop('Rent', axis=1)
y = data['Rent']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [209]:
# Нормализуем данные
scaler = StandardScaler()
sc_y = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Найдем решение задачи в замкнутом виде с помощью псевдообратной матрицы.
Для этого воспользуемся функциями из numpy, а позже получим те же значения с помощью функций из sklearn для сравнения

In [210]:
# Найдем решение с помощью псевдообратной матрицы для OLS
def closed_form_ols(X, y):
    X_transpose = np.transpose(X)
    X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X))
    X_transpose_y = np.dot(X_transpose, y)
    theta = np.dot(X_transpose_X_inv, X_transpose_y)
    return theta

theta_ols = closed_form_ols(X_train, y_train)
print(theta_ols)

[  2718.71400164  24252.54009331   9483.51722303    112.77729514
  -8911.87925602  -2583.69311496  -3204.69861232  -5315.75091805
   1991.97310378 -10307.56314367  -2029.91832984  18775.40912466
   2064.75499528  -1175.8199536    -812.73796294   -802.5264888
  -1136.57379652  -4161.14316176  -2849.73755441    354.45674058
  -6900.31629318]


In [211]:
# Найдем решение с помощью псевдообратной матрицы для Ridge
def closed_form_ridge(X, y, alpha):
    X_transpose = np.transpose(X)
    identity = np.eye(X.shape[1])
    X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X) + alpha * identity)
    X_transpose_y = np.dot(X_transpose, y)
    theta = np.dot(X_transpose_X_inv, X_transpose_y)
    return theta

theta_ridge = closed_form_ridge(X_train, y_train, alpha=0.1)
print(theta_ridge)

[ 3078.33787597 23243.00165847 10195.33294289   190.96017714
   746.19018834  -754.93343456 -2334.65454623 -5385.25080894
   433.16182566 -8751.39617575 -2749.9417648  17614.66879448
  2332.46827683  -962.85094129  -684.12523805  1029.32061116
   862.03200272 -2607.58937062  2185.11226706   471.88398607
 -2201.03047559]


In [212]:
def gradient_descent(X, y, theta_init, learning_rate, num_iterations):
    theta = theta_init.copy()
    m = len(y)
    for iteration in range(num_iterations):
        error = np.dot(X, theta) - y
        gradient = (1/m) * np.dot(X.T, error)
        theta = theta - learning_rate * gradient  # Reshape the gradient array
    return theta

X_train_with_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_with_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

theta_init = np.zeros(X_train_with_bias.shape[1])  # Инициализируем нулями
learning_rate = 0.01 
num_iterations = 1000  

theta_gradient_descent = gradient_descent(X_train_with_bias, y_train, theta_init, learning_rate, num_iterations)

print(theta_gradient_descent)

[35149.99879817  3694.49054164 22071.0631258  10636.68360586
   194.33126011   699.39394668  -708.29357239 -2191.76966446
 -5315.26198664   279.8805297  -8599.96006643 -2704.41769419
 17355.07303614  2374.93540232  -965.79550268  -711.55790505
  1048.38112454   855.47558035 -2622.13515646  2311.06663242
   464.78900678 -2326.71140878]


In [213]:
#Посмотрим на полученные модели
n = X.shape[1] - 1
linear_function_ols = "y = "
for i in range(n):
    linear_function_ols += f"{theta_ols[i]} * x{i+1} + "
linear_function_ols += f"{theta_ols[n]}"
print("Линейная регрессия:", linear_function_ols)


linear_function_ridge = "y = "
for i in range(n):
    linear_function_ridge += f"{theta_ridge[i]} * x{i+1} + "
linear_function_ridge += f"{theta_ridge[n]}"
print("Ridge (L2):", linear_function_ridge)


linear_function_gradient_descent = "y = "
for i in range(n):
    linear_function_gradient_descent += f"{theta_gradient_descent[i]} * x{i+1} + "
linear_function_gradient_descent += f"{theta_gradient_descent[n]}"
print("Градиентный спуск:", linear_function_gradient_descent)


Линейная регрессия: y = 2718.714001636131 * x1 + 24252.54009331136 * x2 + 9483.517223028866 * x3 + 112.77729514396437 * x4 + -8911.879256019412 * x5 + -2583.693114956183 * x6 + -3204.6986123241763 * x7 + -5315.7509180525185 * x8 + 1991.9731037793838 * x9 + -10307.56314366815 * x10 + -2029.9183298350747 * x11 + 18775.409124657053 * x12 + 2064.75499527563 * x13 + -1175.8199536016793 * x14 + -812.7379629442964 * x15 + -802.5264887981089 * x16 + -1136.5737965210628 * x17 + -4161.143161760814 * x18 + -2849.7375544145525 * x19 + 354.4567405835712 * x20 + -6900.316293178943
Ridge (L2): y = 3078.337875965751 * x1 + 23243.001658472214 * x2 + 10195.332942889905 * x3 + 190.96017714071272 * x4 + 746.1901883434894 * x5 + -754.9334345649445 * x6 + -2334.654546226496 * x7 + -5385.250808938496 * x8 + 433.16182566212245 * x9 + -8751.396175750846 * x10 + -2749.9417648006224 * x11 + 17614.668794482597 * x12 + 2332.468276826155 * x13 + -962.8509412945313 * x14 + -684.1252380452851 * x15 + 1029.32061116199

In [214]:

def print_metrics_for_theta(X_test,theta):
    y_pred = np.dot(X_test, theta)
    mae = np.mean(np.abs(y_pred - y_test))
    mse = np.mean((y_pred - y_test) ** 2)
    rmse = np.sqrt(mse)
    print("Mean Absolute Error (MAE):", mae)
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("\n")

print("Линейная регрессия")
print_metrics_for_theta(X_test, theta_ols)
print("Ridge (L2)")
print_metrics_for_theta(X_test, theta_ridge)
print("Градиентный спуск")
print_metrics_for_theta(X_test_with_bias, theta_gradient_descent)


Линейная регрессия
Mean Absolute Error (MAE): 36946.69564637552
Mean Squared Error (MSE): 3154829916.4354167
Root Mean Squared Error (RMSE): 56167.87263583531


Ridge (L2)
Mean Absolute Error (MAE): 36870.67660011662
Mean Squared Error (MSE): 3105682758.908532
Root Mean Squared Error (RMSE): 55728.65294360283


Градиентный спуск
Mean Absolute Error (MAE): 22171.447982042526
Mean Squared Error (MSE): 1920221117.704339
Root Mean Squared Error (RMSE): 43820.327676825276




### Какая модель имеет наименьшее значение функции потерь на тестовой выборке?


**Наименьшее значение функции потерь MSE на тестовой выборке имеет модель, полученная градиентным спуском, следовательно, она имеет наилучшую точность**


Значение функции потерь для случая с L2 регуляцией меньше, чем без него. Регуляризация помогает избежать эффекта переобучения в данном примере. Использование L2-регуляризации (Ridge) позволяет штрафовать большие значения коэффициентов модели, что помогает уменьшить переобучение.

### Моделирование


In [215]:
from numpy import ndarray
# Модель линейной регрессии без регуляризации
ols_linear_reg = LinearRegression()
ols_linear_reg.fit(X_train, y_train)
ols_linear_reg_prediction = ols_linear_reg.predict(X_test)

def print_metrics(prediction: ndarray):
    # Evaluation metrics
    mae = metrics.mean_absolute_error(y_test, prediction)
    mse=  metrics.mean_squared_error(y_test, prediction)
    rmse =  np.sqrt(mse)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('\n')

print("Значения функции потерь (линейная регрессия):")
print_metrics(ols_linear_reg_prediction)


Значения функции потерь (линейная регрессия):
MAE: 22148.57412587433
MSE: 1912217685.5071487
RMSE: 43728.91132314123




In [216]:
# Модель линейной регрессии с L2-регуляризацией (Ridge)
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(X_train, y_train)
ridge_reg_prediction = ridge_reg.predict(X_test)
print("Значения функции потерь (регуляризация Ridge):")
print_metrics(ridge_reg_prediction)

Значения функции потерь (регуляризация Ridge):
MAE: 22231.23802532524
MSE: 1911230601.2362475
RMSE: 43717.62346281243




Получим модель линейной регрессии с помощью стохастического градиентного спуска

In [217]:
sgdr_reg = SGDRegressor(tol=.0001, eta0=.01) 
sgdr_reg.fit(X_train, y_train)
sgdr_reg_prediction = sgdr_reg.predict(X_test)
print("Значения функции потерь (Стохастический градиентный спуск):")
print_metrics(sgdr_reg_prediction)

n = X.shape[1] - 1

# Найдем градиент
theta = sgdr_reg.coef_ # коэффициенты
m = len(X_train)  # количество характеристик

gradient = (1/m) * np.dot(X_train.T, (np.dot(X_train, theta) - y_train))

print(f"Градиент:\n {gradient}")

Значения функции потерь (Стохастический градиентный спуск):
MAE: 67556.44778540895
MSE: 9254697805.888643
RMSE: 96201.3399381144


Градиент:
 [-67805.62804351 -53887.5618814  -62354.71336614   1523.69546816
 -53456.44543932  53384.66792072 -18645.52095505 -20913.45160433
 -11985.19469922  76273.79714109  -5753.73858863 -19813.35097088
 -17185.03899769 -48272.42950025  61890.43733709 -25366.18094467
  35937.77299339 -21414.32697808 -32118.96863782 896515.05273461
    978.40049224]


### Какие признаки оказывают наибольший вклад в точность определения стоимости аренды?

Чтобы определить признаки, которые оказывают наибольший вклад в точность определения стоимости аренды, можно использовать коэффициенты модели линейной регрессии. Чем больше абсолютное значение коэффициента, тем больший вклад вносит соответствующий признак.

In [218]:
# Выведем все коэффициенты от самого высокого по модулю до самого низкого (т.е. от более значимых до менее значимых) для каждой из представленных моделей
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ols_linear_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для линейной регрессии:")
print(feature_importance_sorted)

Наиболее важные признаки для линейной регрессии:
                              Признак   Коэффициент
5                Area Type_Super Area  2.203124e+17
4               Area Type_Carpet Area  2.203049e+17
20     Point of Contact_Contact Owner  1.284339e+17
18     Point of Contact_Contact Agent  1.284063e+17
16  Tenant Preferred_Bachelors/Family -8.776834e+16
11                        City_Mumbai  8.528406e+16
6                      City_Bangalore  8.337818e+16
7                        City_Chennai  8.289407e+16
9                      City_Hyderabad  8.135026e+16
15         Tenant Preferred_Bachelors -7.489404e+16
8                          City_Delhi  7.094643e+16
10                       City_Kolkata  6.545185e+16
17            Tenant Preferred_Family -5.857866e+16
3                Area Type_Built Area  1.011634e+16
13   Furnishing Status_Semi-Furnished  7.983530e+15
14      Furnishing Status_Unfurnished  7.775199e+15
12        Furnishing Status_Furnished  5.576135e+15
19   Point of C

In [219]:

feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ridge_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для Ridge:")
print(feature_importance_sorted)

Наиболее важные признаки для Ridge:
                              Признак   Коэффициент
1                                Size  23236.682368
11                        City_Mumbai  17611.681952
2                            Bathroom  10196.250431
9                      City_Hyderabad  -8749.633215
7                        City_Chennai  -5384.293605
0                                 BHK   3081.155265
10                       City_Kolkata  -2749.625945
17            Tenant Preferred_Family  -2607.177939
6                      City_Bangalore  -2333.885349
12        Furnishing Status_Furnished   2332.687384
20     Point of Contact_Contact Owner  -2202.134926
18     Point of Contact_Contact Agent   2186.221892
15         Tenant Preferred_Bachelors   1029.110425
13   Furnishing Status_Semi-Furnished   -962.767640
16  Tenant Preferred_Bachelors/Family    861.936758
5                Area Type_Super Area   -755.017416
4               Area Type_Carpet Area    746.276894
14      Furnishing Status_Un

In [220]:
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": sgdr_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для градиентного спуска:")
print(feature_importance_sorted)

Наиболее важные признаки для градиентного спуска:
                              Признак    Коэффициент
19   Point of Contact_Contact Builder  892285.770939
9                      City_Hyderabad   31294.751366
20     Point of Contact_Contact Owner  -30016.835819
14      Furnishing Status_Unfurnished   20121.186770
0                                 BHK  -18685.096358
13   Furnishing Status_Semi-Furnished  -16085.447465
7                        City_Chennai  -15598.806523
4               Area Type_Carpet Area  -14199.652589
5                Area Type_Super Area   14181.093571
6                      City_Bangalore  -10958.985362
11                        City_Mumbai   10506.085448
10                       City_Kolkata  -10338.551732
16  Tenant Preferred_Bachelors/Family    9291.582842
1                                Size    8449.134395
8                          City_Delhi   -7870.328010
2                            Bathroom   -7445.441430
17            Tenant Preferred_Family   -6423.413